In [ ]:
from classes.EventFactory import Event, EventFactoryfrom classes.TargetFactory import Target, TargetFactoryfrom classes.Segment import Segmentfrom modules import SegmentGenerationV2from modules import FillSegmentsfrom classes.StoryTree import StoryPoolfrom modules import Loadfrom classes.VectorStore import VectorStore#Initialize factoriesvideo_name = "accident1"event_factory = EventFactory.from_config(f"{video_name}/configs/event_factory_config.json")target_factory = TargetFactory.from_config(f"{video_name}/configs/target_factory_config.json")#Special pack: overlapping segments with 20% overlap。segments = SegmentGenerationV2.generate_segments(f"{video_name}/frames_cache", frames_per_segment=20, overlap_ratio= 0.2)# Fill segments from JSON filesFillSegments.fill_segments(segments, event_factory, target_factory, video_name)# Build story poolstory_pool = StoryPool(segments)# Load as LangChain RAG documentsstory_docs = Load.load_stories(story_pool, use_summary=True)event_docs = Load.load_events(segments)target_docs = Load.load_targets(segments)segment_docs = Load.load_segments(segments)# Build vector storevector_store = VectorStore()vector_store.build_by_param(segment_docs=segment_docs, event_docs=event_docs, target_docs=target_docs, story_docs=story_docs)

In [ ]:
# Search storylinesquery = "traffic accident"filter = {"type": "accident"}story_results = vector_store.search_story(query=query, top_k=3, rerank=True, filter=filter)for result in story_results:print(result.page_content)print("=========================================")for result in story_results:print(result.metadata.get("whole_story"))print("=========================================")

In [ ]:
# Search eventsquery = "accident"event_results = vector_store.search_event(query=query, top_k=3)for result in event_results:print(result.page_content)

In [ ]:
# Search targetsquery = "accident"target_results = vector_store.search_target(query=query, top_k=3)for result in target_results:print(result.page_content)

In [ ]:
# Search segmentsquery = "accident"segment_results = vector_store.search_segment(query=query, top_k=3)for result in segment_results:print(result.page_content)

In [ ]:
# Assistantfrom classes.RAGAgent import Agentagent = Agent(vectorstore=vector_store, story_pool=story_pool, segments=segments, event_factory=event_factory, target_factory=target_factory)

In [ ]:
# Smart searchquery = "traffic accident"mode = ""result = agent.search(query=query, mode=mode)

In [ ]:
for message in result.get("messages")[1:]:message.pretty_print()